In [144]:
import os
import json
import sqlite3
import pandas as pd
import datetime

In [145]:
all_events = []
for root,directories, files in os.walk('../nni'):
    for file in files:
        file_name = '{}/{}'.format(root,file)
        meta_data = file_name.split('/')
        experiment_id = meta_data[3]
        if 'log' in file_name or 'run.sh' in file_name or 'stderr' in file_name:
            continue
        if 'sqlite' in file_name:
                try:
                    conn = sqlite3.connect(file_name)
                    c = conn.cursor()
                    c.execute('SELECT * FROM TrialJobEvent')
                    result = c.fetchall()
                    all_events+=[{'time_point':  datetime.datetime.fromtimestamp(i[0]/1000),
                                  'experiment_id': experiment_id,
                                  'trial_id': i[1],
                                  'type': i[2],
                                  'queue': json.loads(i[3])['parameter_id'] if i[3] else None,
                                  'batch': json.loads(i[3])['parameters']['batch'] if i[3] else None,
                                  'hidden_factors': json.loads(i[3])['parameters']['hidden_factors'] if i[3] else None,
                                  'regularizer': json.loads(i[3])['parameters']['regularizer'] if i[3] else None,
                                 } for i in result] 
                finally:
                    conn.close()

In [146]:
all_events_df = pd.DataFrame(all_events)
all_events_df['duration'] = 0

In [147]:
for trial_id, body in all_events_df.groupby('trial_id'):
    begin_time = body[body['type']=='RUNNING']['time_point'].array
    end_time = body[body['type']=='SUCCEEDED']['time_point'].array
    if end_time.size == 0:
        all_events_df = all_events_df[all_events_df['trial_id']!=trial_id]
    else:
        all_events_df.loc[all_events_df['trial_id']==trial_id, 'duration'] = (end_time[0] - begin_time[0]).total_seconds()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [148]:
all_events_df = all_events_df[all_events_df['type']=='WAITING'].drop(columns=['type'])
all_events_df['value'] = 0
all_events_df

,time_point,experiment_id,trial_id,queue,batch,hidden_factors,regularizer,duration,value
0,2020-06-30 17:43:06.738,MrUaVQmO,nwxKc,3_0_0,512.0,8.0,-4.533964,154.509,0
3,2020-06-30 17:45:52.015,MrUaVQmO,KUJca,3_0_1,1024.0,8.0,-6.233733,95.362,0
6,2020-06-30 17:47:42.251,MrUaVQmO,XF4ZS,3_0_2,1024.0,8.0,-6.658249,94.922,0
9,2020-06-30 17:49:22.439,MrUaVQmO,w1Zey,3_0_3,1024.0,32.0,-6.754749,99.247,0
12,2020-06-30 17:51:12.637,MrUaVQmO,J06s9,3_0_4,2048.0,128.0,-4.749582,85.147,0
...,...,...,...,...,...,...,...,...,...
1064,2020-06-26 14:10:42.557,SreqCz44,RMHeM,3_0_25,2048.0,64.0,-7.350540,72.895,0
1067,2020-06-26 14:12:02.736,SreqCz44,wC9Up,3_0_26,2048.0,64.0,-7.530508,72.526,0
1070,2020-06-26 14:13:22.876,SreqCz44,TDSw3,3_1_4,128.0,32.0,-6.690079,475.677,0
1073,2020-06-26 14:21:28.934,SreqCz44,WVDEn,3_1_5,128.0,128.0,-4.879490,542.293,0


In [149]:
with open('experiment_model1.json', 'r') as f:
    data_json = json.loads(f.read())

In [152]:
for k, v in data_json.items():
    for trial in v['trials']:
        if 'trial_job_id' not in trial:
            continue
        all_events_df.loc[all_events_df['trial_id']==trial['trial_job_id'], 'value'] = trial['value']

In [153]:
all_events_df

,time_point,experiment_id,trial_id,queue,batch,hidden_factors,regularizer,duration,value
0,2020-06-30 17:43:06.738,MrUaVQmO,nwxKc,3_0_0,512.0,8.0,-4.533964,154.509,0.5722249746322632
3,2020-06-30 17:45:52.015,MrUaVQmO,KUJca,3_0_1,1024.0,8.0,-6.233733,95.362,0.49950000643730164
6,2020-06-30 17:47:42.251,MrUaVQmO,XF4ZS,3_0_2,1024.0,8.0,-6.658249,94.922,0.49985000491142273
9,2020-06-30 17:49:22.439,MrUaVQmO,w1Zey,3_0_3,1024.0,32.0,-6.754749,99.247,0.49912500381469727
12,2020-06-30 17:51:12.637,MrUaVQmO,J06s9,3_0_4,2048.0,128.0,-4.749582,85.147,0.49729999899864197
...,...,...,...,...,...,...,...,...,...
1064,2020-06-26 14:10:42.557,SreqCz44,RMHeM,3_0_25,2048.0,64.0,-7.350540,72.895,0.49729999899864197
1067,2020-06-26 14:12:02.736,SreqCz44,wC9Up,3_0_26,2048.0,64.0,-7.530508,72.526,0.49729999899864197
1070,2020-06-26 14:13:22.876,SreqCz44,TDSw3,3_1_4,128.0,32.0,-6.690079,475.677,0.7490749955177307
1073,2020-06-26 14:21:28.934,SreqCz44,WVDEn,3_1_5,128.0,128.0,-4.879490,542.293,0.7528499960899353
